<a href="https://colab.research.google.com/github/icaro-bezerra/projeto-01/blob/main/notebooks/01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.0 Importação de Dados

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import sqlite3
import pandas as pd
import numpy as np
conn = sqlite3.connect("/content/drive/MyDrive/data_source/database.db")

In [52]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('flight_activity',), ('flight_loyalty_history',)]


In [53]:
query = """
  SELECT
    *
  FROM flight_activity fa
    LEFT JOIN flight_loyalty_history fl
  ON (fa.loyalty_number = fl.loyalty_number)
"""
df = pd.read_sql_query(query, conn)
df

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405619,999902,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Aurora,7290.07,Standard,2014,5,NaN,NaN
405620,999911,2018,12,0,0,0,0,0.0,0,0,...,Doctor,217943.0,Single,Nova,8564.77,Standard,2012,8,NaN,NaN
405621,999940,2018,12,3,0,3,1233,123.0,0,0,...,Bachelor,47670.0,Married,Nova,20266.50,Standard,2017,7,NaN,NaN
405622,999982,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Star,2631.56,Standard,2018,7,NaN,NaN


# 2.0 Preparação e inspeção de Dados

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loyalty_number               405624 non-null  int64  
 1   year                         405624 non-null  int64  
 2   month                        405624 non-null  int64  
 3   flights_booked               405624 non-null  int64  
 4   flights_with_companions      405624 non-null  int64  
 5   total_flights                405624 non-null  int64  
 6   distance                     405624 non-null  int64  
 7   points_accumulated           405624 non-null  float64
 8   points_redeemed              405624 non-null  int64  
 9   dollar_cost_points_redeemed  405624 non-null  int64  
 10  loyalty_number               405624 non-null  int64  
 11  country                      405624 non-null  object 
 12  province                     405624 non-null  object 
 13 

In [55]:
mem_by_col = df.memory_usage(deep=True)
mem_by_col

Index                               128
loyalty_number                  3244992
year                            3244992
month                           3244992
flights_booked                  3244992
flights_with_companions         3244992
total_flights                   3244992
distance                        3244992
points_accumulated              3244992
points_redeemed                 3244992
dollar_cost_points_redeemed     3244992
loyalty_number                  3244992
country                        25554312
province                       27096936
city                           26423832
postal_code                    25959936
gender                         25150344
education                      26430600
salary                          3244992
marital_status                 25912464
loyalty_card                   24910728
clv                             3244992
enrollment_type                26506104
enrollment_year                 3244992
enrollment_month                3244992


In [56]:
totalMem = df.memory_usage(deep=True).sum() / (1024 ** 3)
totalMem

0.2692549005150795

In [57]:
# cleaning foreign key
df = df.loc[:,~df.columns.duplicated()]


In [58]:
df.loc[:, df.dtypes == np.object].keys()

<ipython-input-58-44c8111be486>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df.loc[:, df.dtypes == np.object].keys()


Index(['country', 'province', 'city', 'postal_code', 'gender', 'education',
       'marital_status', 'loyalty_card', 'enrollment_type'],
      dtype='object')

In [59]:
df['country'] = df['country'].astype('category')

<ipython-input-59-88dd4666e518>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['country'] = df['country'].astype('category')


In [60]:
df['province'] = df['province'].astype('category')

<ipython-input-60-fff3cd89f986>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['province'] = df['province'].astype('category')


In [61]:
df['city'] = df['city'].astype('category')

<ipython-input-61-09fb20750947>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['city'] = df['city'].astype('category')


In [62]:
df['postal_code'] = df['postal_code'].astype('category')

<ipython-input-62-ff7609dd378e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['postal_code'] = df['postal_code'].astype('category')


In [63]:
df['gender'] = df['gender'].astype('category')

<ipython-input-63-1f3f2b0b086e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'] = df['gender'].astype('category')


In [64]:
df['education'] = df['education'].astype('category')

<ipython-input-64-ef46037cde1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['education'] = df['education'].astype('category')


In [65]:
df['marital_status'] = df['marital_status'].astype('category')

<ipython-input-65-5776af650d86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['marital_status'] = df['marital_status'].astype('category')


In [66]:
df['enrollment_type'] = df['enrollment_type'].astype('category')

<ipython-input-66-5727b807e975>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['enrollment_type'] = df['enrollment_type'].astype('category')


In [67]:
df['loyalty_card'] = df['loyalty_card'].astype('category')

<ipython-input-67-1a805a66939c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loyalty_card'] = df['loyalty_card'].astype('category')


In [68]:
df.to_parquet('flights.parquet')

In [69]:
df = pd.read_parquet('flights.parquet')

In [70]:
nullByColumn = df.isna().sum()
nullByColumn

loyalty_number                      0
year                                0
month                               0
flights_booked                      0
flights_with_companions             0
total_flights                       0
distance                            0
points_accumulated                  0
points_redeemed                     0
dollar_cost_points_redeemed         0
country                             0
province                            0
city                                0
postal_code                         0
gender                              0
education                           0
salary                         102672
marital_status                      0
loyalty_card                        0
clv                                 0
enrollment_type                     0
enrollment_year                     0
enrollment_month                    0
cancellation_year              355560
cancellation_month             355560
dtype: int64

In [71]:
# num_columns = df.select_dtypes(include=np.number).keys()

num_columns = ['year', 'month', 'flights_booked', 'flights_with_companions', 'total_flights', 'distance',
                  'points_accumulated', 'salary', 'clv', 'loyalty_card']

num_columns = df.loc[:,num_columns]
clean_data = num_columns.dropna()
clean_data.isna().sum()

year                       0
month                      0
flights_booked             0
flights_with_companions    0
total_flights              0
distance                   0
points_accumulated         0
salary                     0
clv                        0
loyalty_card               0
dtype: int64

In [72]:
clean_data.shape[0]

302952

# 3.0 Exercicios

In [73]:
query = """
  SELECT
    fa.loyalty_number,
    fa.year,
    fa.month,
    fa.flights_booked
  FROM flight_activity fa
  WHERE
    fa.flights_booked > 3
"""
df.tail()


,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
405619,999902,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Aurora,7290.07,Standard,2014,5,NaN,NaN
405620,999911,2018,12,0,0,0,0,0.0,0,0,...,Doctor,217943.0,Single,Nova,8564.77,Standard,2012,8,NaN,NaN
405621,999940,2018,12,3,0,3,1233,123.0,0,0,...,Bachelor,47670.0,Married,Nova,20266.50,Standard,2017,7,NaN,NaN
405622,999982,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Star,2631.56,Standard,2018,7,NaN,NaN
405623,999986,2018,12,0,0,0,0,0.0,0,0,...,Bachelor,46594.0,Married,Nova,8257.01,2018 Promotion,2018,2,NaN,NaN


In [74]:
query = """
  SELECT
    fa.loyalty_number,
    fa.year,
    fa.month,
    fa.flights_booked,
    fa.total_flights,
    fa.distance,
    fa.points_accumulated
  FROM flight_activity fa
"""
df


,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405619,999902,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Aurora,7290.07,Standard,2014,5,NaN,NaN
405620,999911,2018,12,0,0,0,0,0.0,0,0,...,Doctor,217943.0,Single,Nova,8564.77,Standard,2012,8,NaN,NaN
405621,999940,2018,12,3,0,3,1233,123.0,0,0,...,Bachelor,47670.0,Married,Nova,20266.50,Standard,2017,7,NaN,NaN
405622,999982,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Star,2631.56,Standard,2018,7,NaN,NaN


In [75]:
query = """
  SELECT
    fa.loyalty_number,
    fa.year,
    fa.month,
    fa.flights_booked,
    fa.total_flights,
    fa.distance,
    fa.points_accumulated
  FROM flight_activity fa
  WHERE
    fa.distance > 2000
"""
df

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405619,999902,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Aurora,7290.07,Standard,2014,5,NaN,NaN
405620,999911,2018,12,0,0,0,0,0.0,0,0,...,Doctor,217943.0,Single,Nova,8564.77,Standard,2012,8,NaN,NaN
405621,999940,2018,12,3,0,3,1233,123.0,0,0,...,Bachelor,47670.0,Married,Nova,20266.50,Standard,2017,7,NaN,NaN
405622,999982,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Star,2631.56,Standard,2018,7,NaN,NaN


In [76]:
query = """
  SELECT
    fa.loyalty_number,
    fa.year,
    fa.month,
    fa.flights_booked,
    fa.total_flights,
    fa.distance,
    fa.points_accumulated
  FROM flight_activity fa
  WHERE
    fa.distance > 2000
  AND
    fa.month = 9
"""
df

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405619,999902,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Aurora,7290.07,Standard,2014,5,NaN,NaN
405620,999911,2018,12,0,0,0,0,0.0,0,0,...,Doctor,217943.0,Single,Nova,8564.77,Standard,2012,8,NaN,NaN
405621,999940,2018,12,3,0,3,1233,123.0,0,0,...,Bachelor,47670.0,Married,Nova,20266.50,Standard,2017,7,NaN,NaN
405622,999982,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Star,2631.56,Standard,2018,7,NaN,NaN


In [77]:
query = """
  SELECT
    fa.loyalty_number,
    fa.year,
    fa.month,
    fa.flights_booked,
    fa.total_flights,
    fa.distance,
    fa.points_accumulated
  FROM flight_activity fa
  WHERE
    fa.distance > 2000
  OR
    fa.points_accumulated < 100
"""
df

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405619,999902,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Aurora,7290.07,Standard,2014,5,NaN,NaN
405620,999911,2018,12,0,0,0,0,0.0,0,0,...,Doctor,217943.0,Single,Nova,8564.77,Standard,2012,8,NaN,NaN
405621,999940,2018,12,3,0,3,1233,123.0,0,0,...,Bachelor,47670.0,Married,Nova,20266.50,Standard,2017,7,NaN,NaN
405622,999982,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Star,2631.56,Standard,2018,7,NaN,NaN


In [78]:
query = """
  SELECT
    *
  FROM flight_loyalty_history fl

  WHERE
    fl.loyalty_card = 'Star'
"""
df

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405619,999902,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Aurora,7290.07,Standard,2014,5,NaN,NaN
405620,999911,2018,12,0,0,0,0,0.0,0,0,...,Doctor,217943.0,Single,Nova,8564.77,Standard,2012,8,NaN,NaN
405621,999940,2018,12,3,0,3,1233,123.0,0,0,...,Bachelor,47670.0,Married,Nova,20266.50,Standard,2017,7,NaN,NaN
405622,999982,2018,12,0,0,0,0,0.0,0,0,...,College,NaN,Married,Star,2631.56,Standard,2018,7,NaN,NaN


In [79]:
query = """
  SELECT
    *
  FROM flight_activity fa
"""
df.head()


,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN


# 4.0 Treino de Modelo


In [80]:
clean_data.max()
clean_data.min()


<ipython-input-80-2feb6285921d>:1: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  clean_data.max()
<ipython-input-80-2feb6285921d>:2: FutureWarning: The default value of numeric_only in DataFrame.min is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  clean_data.min()


year                        2017.00
month                          1.00
flights_booked                 0.00
flights_with_companions        0.00
total_flights                  0.00
distance                       0.00
points_accumulated             0.00
salary                    -58486.00
clv                         2119.89
dtype: float64

In [81]:
from sklearn import tree as tr

X_attributes = clean_data.drop(columns = 'loyalty_card')
Y_labels = clean_data['loyalty_card']

# algo definition
model = tr.DecisionTreeClassifier(max_depth=5)

model.fit(X_attributes, Y_labels)

DecisionTreeClassifier(max_depth=5)

In [82]:
tr.plot_tree(model, filled = True);

# 5.0 Apresentando Resultado

In [83]:
sample = X_attributes.sample()
sample

,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,salary,clv
96420,2017,6,0,0,0,0,0.0,75537.0,9156.32


In [84]:
# sample = X_attributes.sample()
proj = model.predict_proba(sample)
print('''
Projection
Aurora {:.1f}%
Nova {:.1f}%
Start {:.1f}%
'''
.format(100*proj[0][0], 100*proj[0][1], 100*proj[0][2]))


Projection 
Aurora 31.2%
Nova 35.1%
Start 33.8%



# 6.0 Painel de Visualização

In [85]:
!pip install gradio
import gradio as gr

In [ ]:
def project(*args):
  X_attributes = np.array([args]).reshape(1, -1)
  proj = model.predict_proba(X_attributes)
  return {
      "Aurora":proj[0][0],
      "Nova":proj[0][1],
      "Star":proj[0][2]
  }

with gr.Blocks() as demo:
  gr.Markdown(''' # Purchasing Propensity''')

  with gr.Row():
    with gr.Column():
      gr.Markdown(''' # Client Attributes''')
      year	=                     gr.Slider(label="year", minimum = 2017, maximum = 2018, step = 1, randomize = True)
      month =                     gr.Slider(label="month", minimum = 1, maximum = 12, step = 1, randomize = True)
      flights_booked	=           gr.Slider(label="flights booked", minimum = 0, maximum = 21, step = 1, randomize = True)
      flights_with_companions	=   gr.Slider(label="flights with companions", minimum = 0, maximum = 11, step = 1, randomize = True)
      total_flights	=             gr.Slider(label="total flights", minimum = 0, maximum = 32, step = 1, randomize = True)
      distance	=                 gr.Slider(label="distance", minimum = 0, maximum = 6293, step = 1, randomize = True)
      points_accumulated	=       gr.Slider(label="points accumulated", minimum = 0.00, maximum = 676.50, step = 1, randomize = True)
      salary	=                   gr.Slider(label="salary", minimum = -58486.00, maximum = 407228.00, step = 1, randomize = True)
      clv =                       gr.Slider(label="clv", minimum =  2119.89, maximum = 83325.38, step = 1, randomize = True)
      with gr.Row():

        with gr.Row():

          project_btn = gr.Button(value = 'Projection')

    with gr.Column():
      gr.Markdown(''' # COL B''')
      label = gr.Label()

  project_btn.click(
      fn=project,
      inputs = [
          year,
          month,
          flights_booked,
          flights_with_companions,
          total_flights,
          distance,
          points_accumulated,
          salary,
          clv
          ],
      outputs = [
          label
      ])
demo.launch(debug = True, share = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f4f3ac6c3894d3bf00.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
